In [ ]:
#load modules 
import ee
import ipywidgets as widgets
from ipywidgets import HBox
from scripts import gdrive

In [ ]:
# Prevent discovery cache Warning
import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

In [ ]:
#initialize earth engine
ee.Initialize()

In [ ]:
#define constants
NO_INPUT = "No input have been selected, provide them through step 1."
BUTTON_COLOR = 'lightgreen' 

In [ ]:
#functions
def download_to_disk(filename, alerts, country):
    """download the tile to the GEE disk"""
    task_config = {
        'image':alerts,
        'description':filename,
        'scale': 30,
        'region':country.geometry(),
        'maxPixels': 1e10
    }
    
    task = ee.batch.Export.image.toDrive(**task_config)
    task.start()
    
    return task

def get_alerts(country_code, year):
    """ get the alerts from the GLAD project"""
    
    country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', country_code))
    all_alerts  = ee.ImageCollection('projects/glad/alert/UpdResult')
    alerts = all_alerts.select('conf' + year[-2:]).mosaic().clip(country);
    
    return (country, alerts)
    
def launch_task(country_code, year):
    
    #search for the task in task_list
    filename = construct_filename(country_code, year)
    tasks_list = ee.batch.Task.list()
    current_task = ''
    for task in tasks_list:
        if task.config['description'] == filename:
            current_task = task
            break
            
    #launch the task in GEE 
    if current_task == '':
        country, alerts = get_alerts(country_code, year)
        current_task = download_to_disk(filename, alerts, country)
        print ("the task {0} has been launched on your GEE account".format(filename))
        return current_task
    
    #give insight if it's still running 
    if current_task.state == "RUNNING":
        print ("the task {0} is currently running on the GEE servers ....".format(filename))
        return current_task
    
    #say when it's completed
    if current_task.state == "COMPLETED":
        print('the task {0} has already been completed and is available on your Gdrive.'.format(filename))
        return current_task
    
def construct_filename(country_code, year):
    """return the filename associated with the current task"""
    
    filename = 'alerts_' + country_code + '_' + year
    
    return filename

def check_for_inputs():
    """check if the inputs exist and have a value"""
    
    #I just need to check for existance as the first cell has a default value
    return ('country_code' in globals() and 'year' in globals())

In [ ]:
#template
output = widgets.Output()

button_process = widgets.Button(description="Start GEE process!")
button_process.style.button_color = BUTTON_COLOR

In [ ]:
#define the function used to display the process
def display_inputs(country_code, year):
    print ("You have selected the data from {0} for the year {1}.".format(country_code, year))
    
nb_process_click = 0    
def on_button_clicked(b):
    
    global country_code
    global year
    global nb_process_click
    
    
    with output:
        button_process.description = "Check status"
        
        if check_for_inputs():
            if nb_process_click < 1:
                display_inputs(country_code, year)
            launch_task(country_code, year)
            
        else:
            print(NO_INPUT)
            
    nb_process_click += 1

In [1]:
#run
button_process.on_click(on_button_clicked)
iu_process = HBox([button_process])
display(iu_process)
display(output)

NameError: name 'button_process' is not defined